In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import *
from pyspark.ml.regression import *
from pyspark.ml.evaluation import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import seaborn as sns

In [3]:
spark = SparkSession.builder.appName("Vu dep trai").config("spark.executor.memory","12g").getOrCreate()
# conf = pyspark.SparkConf().setMaster("spark://node-master:7077")\
#         .setAppName("Vu dep trai")\
#         .set("spark.executor.memory","15g")
# # sc = SparkContext.getOrCreate(conf=conf)
# # spark.stop()
# sc = SparkContext(conf = conf)
# spark = SparkSession(sc)

In [5]:
target = 'Weekly_Sales'

In [6]:
df_clean = spark.read.csv('train_final.csv', header=True, inferSchema=True)

In [7]:
# setup weight column
df_clean = df_clean.withColumn(
    'Weight',
    when(df_clean.IsHoliday == 1  ,5)\
    .otherwise(1))

In [8]:
# split 80% first data for training
df_train, df_valid = df_clean.randomSplit([0.8, 0.2], seed=1234)
input_col = df_train.columns

In [9]:
## From EDA select important columns
input_col.remove('_c0')
input_col.remove('Weekly_Sales')
input_col.remove('MarkDown1')
input_col.remove('MarkDown2')
input_col.remove('MarkDown3')
input_col.remove('MarkDown4')
input_col.remove('MarkDown5')
input_col.remove('Type')
input_col.remove('Temperature')
input_col.remove('Fuel_Price')
input_col.remove('IsSuperbowl')
input_col.remove('IsLabor')
input_col.remove('IsHoliday')

In [10]:
df_train, df_valid = df_clean.randomSplit([0.8, 0.2], seed=1234)

In [14]:
assembler = VectorAssembler(inputCols=input_col, outputCol="features")

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
# scaler = scaler.fit(df_train)

rf = RandomForestRegressor(featuresCol="scaledFeatures", labelCol="Weekly_Sales", maxDepth=30, numTrees=20, seed=42, maxBins=100000, weightCol = "Weight")
pipeline = pyspark.ml.Pipeline(stages=[assembler,scaler, rf])
model = pipeline.fit(df_train)
# rf = RandomForestRegressor(featuresCol="features", labelCol=target)
# pipeline = pyspark.ml.Pipeline(stages=[assembler, rf])
# paramGrid = pyspark.ml.tuning.ParamGridBuilder()\
#     .addGrid(rf.numTrees, [10])\
#     .addGrid(rf.maxDepth, [20])\
#     .addGrid(rf.maxBins, [100])\
#     .addGrid(rf.featureSubsetStrategy, ["0.8"])\
#     .addGrid(rf.subsamplingRate, [0.2])\
#     .build()
# evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mse")
# crossval = pyspark.ml.tuning.CrossValidator(estimator=pipeline,
#                             estimatorParamMaps=paramGrid,
#                             evaluator=evaluator,
#                             numFolds=3)
# cvModel = crossval.fit(df_train)

Py4JJavaError: ignored

In [ ]:
# bestModel = cvModel.bestModel
# bestModel.stages[-1].extractParamMap()

In [ ]:
train_pred = model.transform(df_train)
valid_pred = model.transform(df_valid)
evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mae", weightCol = "Weight")
train_mae = evaluator.evaluate(train_pred)
valid_mae = evaluator.evaluate(valid_pred)
print("Train MAE: ", train_mae)
print("Valid MAE: ", valid_mae)